# purpose

- fetch open/closed issues and use github issue numbers to:
  - close existing issues
  - edit open issues
  - delete open issues


# structure

- **fetch issues**
  - fetch the last 10 open and closed issues
  - that allows me to close open issues or reopen closed issues
- **dataframe data**
  - make a pandas dataframe from the data fetched from github
  - this is a nested dictionary - each comment and lable is a node inside the corresponding issue
  - so the comments and labels have been exploded into their own separate lines
- **insert data**
  - insert the pandas dataframe into json, csv and excel files
  - this data can then be used for building historical analysis


# setup


In [ ]:
# fetch github token, python packages, queries and other parameters

%run ../../100-set_parameters/100-set_parameters.ipynb # magic commands

# fetch issues


In [ ]:
# query parameters
# specify the query parameters to be used while fetching issues
# github does not allow fetching more than 100 issues

fetch_issue_parameters = {
    "repository_name": "tensorflow",
    "owner_name": "tensorflow",
    "number_of_issues": 100
  }

In [ ]:
# store result

last_few_issues = global_query(fetch_recent_issues, fetch_issue_parameters)

# populate dataframe


## dictionary to dataframe


In [ ]:
# drill down
# take the dictionary result from previous step
# reach out to a particular level within that dictionary (edges in this case)
# store that inside a dataframe

df = pd.DataFrame(d['node'] for d in last_few_issues['data']['repository']['issues']['edges'])

## flatten nested data

- GitHub responds with a nested structure for the query above.
- in this response, each issue may contain various sub-elements, such as labels and comments
- for example, an issue might have the following structure:

```json
{
	"id": "issue-id",
	"title": "Example Issue",
	"labels": [{ "name": "bug" }, { "name": "enhancement" }],
	"comments": [{ "body": "This is a comment." }, { "body": "Another comment." }]
}
```

- in this nested structure, labels and comments are each grouped under a single issue, but for visualizations, we need to flatten this structure or explode it
- if an issue has multiple labels or comments, we want each label or comment to be represented as a separate record associated with the issue
- to achieve this, we can use the following code to flatten the nested data into a DataFrame, enabling easy visualization and analysis:


## comments


In [ ]:
# access comments

df['comments'] = df['comments'].str['edges']

In [ ]:
# flatten comments

# the comments for each issue are nested under the 'comments' key
# code below extracts the nested comments and flatten them into a simpler structure

df_flat_comments = df.explode('comments')
df_flat_comments['comments'] = df_flat_comments['comments'].str['node'].str['body']


## labels


In [ ]:
# access labels
df_flat_comments['labels'] = df_flat_comments['labels'].str['edges']

In [ ]:
# flatten  labels

df_flat_comments_and_labels = df_flat_comments.explode('labels')
df_flat_comments_and_labels['labels'] = df_flat_comments_and_labels['labels'].str['node'].str['name']

## final dataframe


In [ ]:
# print to screen

df_flat_comments_and_labels

# insert data


In [ ]:
# insert into csv

df_flat_comments_and_labels.to_csv("220-fetch_last_few_issues.csv")

In [ ]:
# insert into json

f = open("220-fetch_last_few_issues.json", "w")
f.write(json.dumps(last_few_issues))
f.close()